In [2]:
import pandas as pd
# Concat Stroke codes from St. Lukes and RH for 20223
file_paths = [
    r"C:\Users\nprim\OneDrive\Desktop\Stroke_QI\search__CT SCAN OF THE BRAIN WITHOUT CONTRAST (STROKE)_ _SL CT HEAD_.xlsx",
    r"C:\Users\nprim\OneDrive\Desktop\Stroke_QI\search__CT SCAN OF THE BRAIN WITHOUT CONTRAST (STROKE)_ _RH CT_.xlsx"
]
concatenated_CT_stroke = pd.DataFrame()

for i, file_path in enumerate(file_paths):
    data = pd.read_excel(file_path)
    print(f"Number of studies in doc {i}: {len(data)}")
    concatenated_CT_stroke = pd.concat([concatenated_CT_stroke, data], ignore_index=True)

#output_file_path = 'concatenated_data.xlsx'  

# Write the concatenated DataFrame to a new Excel file
#concatenated_data.to_excel(output_file_path, index=False)

#print(f"Data successfully concatenated and written to {output_file_path}")
#print(f"Total Studies: {len(concatenated_data)}")


Number of studies in doc 0: 729
Number of studies in doc 1: 563


In [ ]:
CT_dizzy = 'CT_dizzy.xlsx'  

# Filter the data for entries where "dizziness" or "dizzy" is mentioned in the "Report Text" column
dizziness_related_studies =concatenated_CT_stroke [extracted_data['Report Text'].str.contains('dizziness|dizzy|lightheaded|lightheadedness', case=False, na=False)]

# Display the filtered data to identify studies ordered for dizziness
dizzy_files = 'dizzy_files.xlsx'
dizziness_related_studies.to_excel(dizzy_files, index=False)
print(f"Number of dizzy studies: {len(dizziness_related_studies)}")


In [6]:
CT_MRN = dizziness_related_studies['Patient MRN']

MRI_brain_df = pd.read_excel(r"C:\Users\nprim\OneDrive\Desktop\Stroke_QI\search.xlsx")

MRI_MRN = MRI_brain_df['Patient MRN']


union_MRN = set(MRI_MRN).intersection(set(CT_MRN))

In [8]:
dizziness_related_studies_with_ct = []
dizziness_related_studies_with_MRI = []
for idx, i in enumerate(dizziness_related_studies['Patient MRN']):
    for j in union_MRN:
        if i == j:
            dizziness_related_studies_with_ct.append(dizziness_related_studies.iloc[idx])
            
dizziness_related_studies_with_ct_df = pd.DataFrame(dizziness_related_studies_with_ct)
dizziness_related_studies_with_MRI_df = pd.DataFrame(dizziness_related_studies_with_MRI)


In [9]:
dizziness_related_studies_with_MRI = []
for idx, i in enumerate(MRI_brain_df['Patient MRN']):
    for j in union_MRN:
        if i == j:
            dizziness_related_studies_with_MRI.append(MRI_brain_df.iloc[idx])
            
dizziness_related_studies_with_MRI_df = pd.DataFrame(dizziness_related_studies_with_MRI)


In [10]:
MRI_df = dizziness_related_studies_with_MRI_df[["Patient MRN", "Scheduled Date"]]

In [13]:
CT_df = dizziness_related_studies_with_ct_df[["Patient MRN", "Scheduled Date"]]

In [14]:
# Rename the 'Scheduled Date' columns for clarity
MRI_df.rename(columns={'Scheduled Date': 'MRI Date'}, inplace=True)
CT_df.rename(columns={'Scheduled Date': 'CT Date'}, inplace=True)

# Merge the two DataFrames on 'Patient MRN'
combined_df = pd.merge(CT_df, MRI_df, on='Patient MRN', how='outer')

C:\Users\nprim\AppData\Local\Temp\ipykernel_19832\525966227.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MRI_df.rename(columns={'Scheduled Date': 'MRI Date'}, inplace=True)
C:\Users\nprim\AppData\Local\Temp\ipykernel_19832\525966227.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CT_df.rename(columns={'Scheduled Date': 'CT Date'}, inplace=True)


In [15]:
combined_df['CT Date'] = pd.to_datetime(combined_df['CT Date'])
combined_df['MRI Date'] = pd.to_datetime(combined_df['MRI Date'])

# Create a new column based on the condition that MRI Date is after CT Date
combined_df['MRI After CT'] = combined_df.apply(lambda row: 'Yes' if row['MRI Date'] > row['CT Date'] else 'No', axis=1)

combined_df

,Patient MRN,CT Date,MRI Date,MRI After CT
0,4082025,2023-08-03 01:10:00,2023-08-03 16:30:00,Yes
1,H250693,2023-06-22 12:15:00,2023-06-22 19:35:00,Yes
2,1005960,2023-11-17 10:40:00,2023-11-18 09:00:00,Yes
3,1005960,2023-11-17 07:15:00,2023-11-18 09:00:00,Yes
4,1005960,2023-11-19 07:20:00,2023-11-18 09:00:00,No
5,2396114,2023-10-29 21:25:00,2023-10-29 19:25:00,No
6,2396114,2023-10-29 12:05:00,2023-10-29 19:25:00,Yes
7,3155702,2023-10-29 06:00:00,2023-10-29 11:30:00,Yes
8,1341510,2023-07-20 02:25:00,2023-07-29 11:30:00,Yes
9,H188080,2023-05-14 16:50:00,2023-05-15 13:25:00,Yes


In [ ]:
import re

# Function to clean up non-human readable sequences from the text
def clean_text(text):
    # Replace specific non-human readable sequences with a space (or with an empty string if preferred)
    cleaned_text = re.sub(r'\n_x000D_', ' ', text)
    cleaned_text = re.sub(r'_x000D_', ' ', text)
    # You can add more replacement rules here if there are other sequences to clean up
    
    # Optional: Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    return cleaned_text.strip()

# Function to extract and clean impression from report text
def extract_impression(report_text):
    try:
        # Split the report text at "Impression:" (case-insensitive)
        parts = re.split(r'(?i)Impression:', report_text, maxsplit=1)
        if len(parts) > 1:
            # Clean up the extracted impression text
            impression_text = clean_text(parts[1])
            return impression_text
        else:
            return 'Impression section not found or not clearly defined.'
    except Exception as e:
        return f'Error extracting impression: {str(e)}'

# Apply the function to extract and clean impressions for each row in the 'Report Text' column
dizziness_related_studies['Impression'] = dizziness_related_studies['Report Text'].apply(extract_impression)

# Display the first few rows to verify the new 'Impression' column
dizziness_related_studies['Impression']

impressions = 'impressions.xlsx'
dizziness_related_studies.to_excel(impressions, index=False)

In [ ]:
for i in range(len(dizziness_related_studies)):
    impression = dizziness_related_studies.iloc[i]['Impression'].lower()
    if ('unremarkable' not in impression):
        if ('no evidence of acute intracranial hemorrhage' not in impression):
            if ('no acute intracranial hemorrhage' not in impression):
                if ('no acute territorial infarct' not in impression):
                    if ('no evidence of intracranial hemorrhage' not in impression):
                        if ('no evidence of acute infarct' not in impression):
                            print(f"{dizziness_related_studies.iloc[i]['Accession Number']}")
                            print(f"{dizziness_related_studies.iloc[i]['Impression']}")

In [ ]:
total_possible_positive_studies = 0
for i in range(len(dizziness_related_studies)):
    impression = dizziness_related_studies.iloc[i]['Impression'].lower()
    conditions = [
        'unremarkable' not in impression,
        'no evidence of acute intracranial hemorrhage' not in impression,
        'no acute intracranial hemorrhage' not in impression,
        'no acute territorial infarct' not in impression,
        'no evidence of intracranial hemorrhage' not in impression,
        'no evidence of acute infarct' not in impression
    ]
    
    if all(conditions):
        accession_number = dizziness_related_studies.iloc[i]['Accession Number']
        print(f"{accession_number}")
        print(impression)
        total_possible_positive_studies +=1

print("Possible Positives", total_possible_positive_studies)
